In [3]:
import numpy as np

In [47]:
class HMM:
    def __init__(self, stateSet, observationSet, transMatrix, obsMatrix, initDist ):
        
        self.stateSet = stateSet
        self.observationSet = observationSet
        
        self.nStates = len(stateSet)
        self.nObs = len(observationSet)    
        
        self.transMatrix = np.array(transMatrix)
        self.obsMatrix = np.array(obsMatrix)
        self.initDist = np.array(initDist)
        
        if(self.transMatrix.shape != (self.nStates, self.nStates)):
            raise ValueError("La matriz de transición debe ser cuadrada de orden número de estados")
            
        if(self.obsMatrix.shape != (self.nStates, self.nObs)):
            raise ValueError("La matriz de emisión debe ser de orden número de estados por número de observaciones")
            
        if(len(self.initDist) != self.nStates):
            raise ValueError("La distribución inicial debe ser un vector de dimensión el número de estados")

        checkStochastic = lambda M: all(np.sum(row)==1.0 for row in M)
        
        if(not checkStochastic(self.transMatrix) or not checkStochastic(self.obsMatrix) or not np.sum(self.initDist)==1.0):
            raise ValueError("Las filas deben sumar 1")
        
    # Parameters:
    # obsIndSeq: secuencia de observaciones indexado    
    def computeAlpha(self, obsIndSeq):
        
        #Calcular α_0(i)
        Alpha=[ [self.initDist[i]*self.nObs[i][obsIndSeq[0]] for i in range(self.nStates) ] ]
        C = [ np.sum(Alpha[0]) ]
        
        
    def viterbi(self, obsIndSeq):
        #Calcular delta_0(i)    
        Delta_ant = np.around([ np.log10( self.obsMatrix[i,obsIndSeq[0]] *self.initDist[i] ) for i in range(self.nStates) ], 4)

        '''
        print(Delta_ant)
        
        [Delta_ant[j] + np.log10(self.transMatrix[j,i]) for j in range(self.nStates) ]


        np.max(np.add( Delta_ant, np.log10(self.transMatrix[:,i])  ))+np.log10(self.obsMatrix[i,obsIndSeq[t]])

        a = np.array( [np.max(  )] for i in range(self.nStates)  )
        b = np.array( [self.initDist[0,i] for i in range(self.nStates)]  )

    
        Delta_ant =  np.around(np.log10( np.multiply( np.array( [self.obsMatrix[i,obsIndSeq[0]] for i in range(self.nStates)]), np.array( [self.initDist[i] for i in range(self.nStates)]  ) )  ) , 4)

        print(Delta_ant)
        Chi = []
        Delta = np.empty(self.nStates)
        subChi = np.empty(self.nStates)

        Delta_ = [Delta_ant]

        for t in range(1, len(obsIndSeq)):
            for i in range(self.nStates):
                Delta[i] = np.max(np.around(np.add( Delta_ant, np.log10(self.transMatrix[:,i])  ), 4)) #+np.log10(self.obsMatrix[i,obsIndSeq[t]])
                subChi[i] = np.argmax(np.add( Delta_ant, np.log10(self.transMatrix[:,i])  ))

            Delta = np.around(np.add(Delta, np.log10(self.obsMatrix[:,obsIndSeq[t]])), 4)
            Delta_.append(Delta)
            Delta_ant = Delta
            Chi.append(subChi)
        '''
        Delta_ = [Delta_ant]
        Chi = []
        for t in range(1, len(obsIndSeq)):
            Delta = np.around([ np.max( np.around([Delta_ant[j] + np.log10(self.transMatrix[j,i]) for j in range(self.nStates) ] ,4) ) + np.log10(self.obsMatrix[i,obsIndSeq[t]]) for i in range(self.nStates) ],4)
            Chi.append( [np.argmax(np.around([Delta_ant[j] + np.log10(self.transMatrix[j,i]) for j in range(self.nStates) ],4)) for i in range(self.nStates) ])
            Delta_ant = Delta
            Delta_.append(Delta)
            
        path = [None] * len(obsIndSeq)
        path[len(obsIndSeq)-1] = np.argmax(Delta)
        
        for t in range(len(obsIndSeq)-2, -1 , -1):
            path[t]=Chi[t][int(path[t+1])]

        return (np.max(Delta), [self.stateSet[int(i)] for i in path], Delta_)

In [48]:
mode = HMM(["-", "+"], [0,1], np.array([[0.9,0.1],[0.1,0.9]]), np.array([[0.4,0.6],[0.75,0.25]]), np.array([0.5,0.5]))

In [49]:
import time
start_time = time.time()
a=mode.viterbi([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1])
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.029651165008544922 seconds ---


In [50]:
[-0.9031+np.log10(0.9),-0.5229+np.log10(0.1)]+np.log10(0.75)

array([-1.07379623, -1.64783874])

In [51]:
print(a[1])
print(a[0])

['+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',

In [52]:
print(a[2])

[array([-0.5229, -0.9031]), array([-0.9666, -1.0738]), array([-1.2342, -1.7217]), array([-1.6779, -1.8924]), array([-2.1216, -2.0631]), array([-2.5653, -2.2338]), array([-2.8329, -2.8817]), array([-3.2766, -3.0524]), array([-3.7203, -3.2231]), array([-4.164 , -3.3938]), array([-4.6077, -3.5645]), array([-4.9624, -3.7352]), array([-5.1331, -3.9059]), array([-5.1277, -4.5538]), array([-5.3953, -5.2017]), array([-5.839 , -5.3724]), array([-6.2827, -5.5431]), array([-6.7264, -5.7138]), array([-6.9356, -6.3617]), array([-7.3793, -6.5324]), array([-7.823 , -6.7031]), array([-8.101 , -6.8738]), array([-8.2717, -7.0445]), array([-8.4424, -7.2152]), array([-8.6131, -7.3859]), array([-8.7838, -7.5566]), array([-8.9545, -7.7273]), array([-8.9491, -8.3752]), array([-9.3928, -8.5459]), array([-9.8365, -8.7166]), array([-10.1145,  -8.8873]), array([-10.2852,  -9.058 ]), array([-10.2798,  -9.7059]), array([-10.5474, -10.3538]), array([-10.9911, -10.5245]), array([-11.4348, -10.6952]), array([-11.8785

In [53]:
b=['+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+',
       '+', '+', '+', '+', '+', '+', '+', '+', '+']

In [54]:
print([a[1][i]==b[i] for i in range(len(a[1]))])

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [55]:
a[1]==b

True